
# Chapter 5 — Interactive Tools (v4, more presets)
**Biofabrication Syllabus (2025)** · *Bioreactors and Tissue Maturation*

New in v4:
- Added **Brain** and **Skin** to tissue presets (alongside Cartilage, Cardiac, Liver).
- Presets auto-fill \(D, k, L\) for diffusion and \(D, k, L, u\) for perfusion helper.


In [ ]:

try:
    from google.colab import output
    output.enable_custom_widget_manager()
except Exception:
    pass


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import (HBox, VBox, Dropdown, Checkbox, Layout,
                        FloatLogSlider, FloatSlider, IntSlider, interactive_output)
from IPython.display import display, Markdown
from math import sqrt

TISSUE_PRESETS = {
    "Cartilage": {"D": 8e-11,  "k": 3e-6,  "L": 2.0e-3, "u": 50e-6},
    "Cardiac":   {"D": 1.2e-10,"k": 6e-6,  "L": 1.0e-3, "u": 150e-6},
    "Liver":     {"D": 1.0e-10,"k": 1e-5,  "L": 1.0e-3, "u": 200e-6},
    "Brain":     {"D": 1.3e-10,"k": 4e-6,  "L": 0.8e-3, "u": 80e-6},
    "Skin":      {"D": 7e-11,  "k": 2e-6,  "L": 1.5e-3, "u": 100e-6},
}



## 1) Fick's Law Visualizer — 1D diffusion with optional consumption

$$
\frac{\partial C}{\partial t} = D\,\frac{\partial^2 C}{\partial x^2} - k\,C
$$


In [ ]:

def simulate_diffusion(D=1e-10, k=5e-6, L=1e-3, C0=1.0, dx=1e-5, t_max=7200, n_curves=5, no_k=False):
    if no_k: k = 0.0
    nx = int(L/dx) + 1
    x  = np.linspace(0, L, nx)
    dt_stable = dx**2/(2*D) if D > 0 else 1.0
    dt = min(0.4*dt_stable, 0.25)
    nsteps = int(np.ceil(t_max/dt))
    t_eval = np.linspace(t_max/n_curves, t_max, n_curves)
    C = np.zeros_like(x); profiles = {}; t = 0.0
    for _ in range(nsteps+1):
        for te in t_eval:
            if abs(t - te) <= 0.5*dt and te not in profiles:
                profiles[te] = C.copy()
        if t >= t_max: break
        d2Cdx2 = np.zeros_like(C)
        d2Cdx2[1:-1] = (C[2:] - 2*C[1:-1] + C[:-2]) / dx**2
        C[0] = C0
        d2Cdx2[-1] = (C[-2] - C[-1]) / dx**2
        C = C + dt*(D*d2Cdx2 - k*C)
        t += dt
    plt.figure()
    for te in sorted(profiles.keys()):
        plt.plot(x*1e3, profiles[te], label=f"t ≈ {int(te)} s")
    plt.xlabel("Depth x (mm)"); plt.ylabel("Concentration C (a.u.)")
    plt.title("Diffusion–reaction profiles in a 1D slab"); plt.legend(); plt.show()
    if k > 0:
        Lp = sqrt(D/k); display(Markdown(f"**Estimated penetration depth:** \(L_p \\approx {Lp*1e3:.2f}\) mm."))
    else:
        display(Markdown("**No consumption (\(k=0\))**: penetration depth grows with \(\\sqrt{D t}\)."))

preset = Dropdown(options=list(TISSUE_PRESETS.keys()), value="Cartilage", description='Preset')
D_slider = FloatLogSlider(value=1e-10, base=10, min=-12, max=-9.0, step=0.1, description='D (m²/s)', layout=Layout(width='340px'))
k_slider = FloatLogSlider(value=5e-6,  base=10, min=-7,  max=-3.5, step=0.1, description='k (1/s)', layout=Layout(width='340px'))
L_slider  = FloatSlider(value=1e-3, min=2e-4, max=5e-3, step=1e-4, description='L (m)', readout_format='.1e')
C0_slider = FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description='C₀ (a.u.)')
dx_slider = FloatSlider(value=1e-5, min=5e-6, max=5e-5, step=5e-6, description='dx (m)', readout_format='.1e')
t_slider  = IntSlider(value=7200, min=600, max=43200, step=600, description='max time (s)')
ncurves   = IntSlider(value=5, min=3, max=8, step=1, description='# curves')
toggle_k0 = Checkbox(value=False, description='No consumption (k=0)')

def apply_preset(change):
    p = TISSUE_PRESETS[preset.value]
    D_slider.value = p["D"]; k_slider.value = p["k"]; L_slider.value = p["L"]
preset.observe(apply_preset, names='value')
apply_preset(None)

ui = VBox([preset, HBox([D_slider, k_slider, toggle_k0]), HBox([L_slider, C0_slider]), HBox([dx_slider, t_slider, ncurves])])
out = interactive_output(simulate_diffusion, {'D':D_slider, 'k':k_slider, 'L':L_slider, 'C0':C0_slider, 'dx':dx_slider, 't_max':t_slider, 'n_curves':ncurves, 'no_k':toggle_k0})
display(ui, out)



## 2) Shear Stress Calculator — rectangular perfusion channel

$$
\tau \approx \frac{6\,\mu\,Q}{b\,h^2}
$$


In [ ]:

from ipywidgets import FloatSlider

def shear_plot(mu=1e-3, Q_mLmin=1.0, b=1e-2, h=5e-4):
    Q = Q_mLmin*1e-6/60.0
    tau = 6.0*mu*Q/(b*h**2)
    Qs = np.linspace(0.1*Q_mLmin, 2.0*Q_mLmin, 60)
    taus = 6.0*mu*(Qs*1e-6/60.0)/(b*h**2)
    plt.figure(); plt.plot(Qs, taus)
    plt.xlabel("Flow rate Q (mL/min)"); plt.ylabel("Wall shear stress τ (Pa)")
    plt.title("Shear stress vs. flow rate"); plt.show()
    display(Markdown(f"**Result:** τ ≈ {tau:.3f} Pa at Q = {Q_mLmin:.2f} mL/min · b = {b*1e3:.1f} mm · h = {h*1e6:.0f} µm."))

ui2 = HBox([
    FloatSlider(value=1e-3, min=0.5e-3, max=2.0e-3, step=0.1e-3, description='μ (Pa·s)'),
    FloatSlider(value=1.0,  min=0.05,   max=5.0,   step=0.05,   description='Q (mL/min)'),
    FloatSlider(value=1e-2, min=2e-3,   max=2e-2,  step=1e-3,   description='b (m)', readout_format='.1e'),
    FloatSlider(value=5e-4, min=1e-4,   max=1e-3,  step=1e-5,   description='h (m)', readout_format='.1e')
])
out2 = interactive_output(shear_plot, {'mu':ui2.children[0], 'Q_mLmin':ui2.children[1], 'b':ui2.children[2], 'h':ui2.children[3]})
display(ui2, out2)



## 3) Perfusion Design Helper — Péclet and Damköhler numbers

$$
\mathrm{Pe} = \frac{uL}{D}, \qquad \mathrm{Da}_{II} = \frac{kL^2}{D}, \qquad \mathrm{Da}_{I} = \frac{kL}{u}
$$


In [ ]:

from ipywidgets import FloatLogSlider

def perfusion_design(D=1e-10, k=5e-6, L=1e-3, u=100e-6):
    Pe  = u*L/D; Da2 = k*L**2/D; Da1 = k*L/u
    display(Markdown(f"**Pe** = {Pe:.2f} &nbsp;&nbsp; **Da₂** = {Da2:.2f} &nbsp;&nbsp; **Da₁** = {Da1:.2f}"))
    us = np.linspace(max(1e-6, 0.2*u), 5*u, 60)
    Pe_s  = us*L/D; Da1_s = k*L/us
    plt.figure(); plt.plot(us*1e6, Pe_s);  plt.xlabel("u (µm/s)");  plt.ylabel("Pe");  plt.title("Effect of u on Pe");  plt.show()
    plt.figure(); plt.plot(us*1e6, Da1_s); plt.xlabel("u (µm/s)");  plt.ylabel("Da₁ = kL/u"); plt.title("Effect of u on Da₁"); plt.show()

D2 = FloatLogSlider(value=1e-10, base=10, min=-12, max=-9.0, step=0.1, description='D (m²/s)')
k2 = FloatLogSlider(value=5e-6,  base=10, min=-7,  max=-3.5, step=0.1, description='k (1/s)')
L2 = FloatSlider(value=1e-3, min=2e-4, max=5e-3, step=1e-4, description='L (m)', readout_format='.1e')
u2 = FloatLogSlider(value=100e-6, base=10, min=-6, max=-2.5, step=0.1, description='u (m/s)')
preset_perf = Dropdown(options=list(TISSUE_PRESETS.keys()), value="Cartilage", description='Preset')

def apply_preset_to_perf(change):
    p = TISSUE_PRESETS[preset_perf.value]
    D2.value = p["D"]; k2.value = p["k"]; L2.value = p["L"]; u2.value = p["u"]

preset_perf.observe(apply_preset_to_perf, names='value')
apply_preset_to_perf(None)

ui3 = VBox([preset_perf, HBox([D2, k2]), HBox([L2, u2])])
out3 = interactive_output(perfusion_design, {'D':D2, 'k':k2, 'L':L2, 'u':u2})
display(ui3, out3)
